<a href="https://colab.research.google.com/github/elichen/oai-playground/blob/main/Batch_processing_with_the_Batch_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install --upgrade openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
from openai import OpenAI
client = OpenAI(api_key=userdata.get('openai-key'))

In [4]:
import json
import pandas as pd
from IPython.display import Image, display

In [6]:
import requests
DATASET_URL = "https://github.com/openai/openai-cookbook/raw/main/examples/data/imdb_top_1000.csv"
response = requests.get(DATASET_URL)
DATASET_PATH = "imdb_top_1000.csv"
with open(DATASET_PATH, 'wb') as file:
    file.write(response.content)

In [7]:
df = pd.read_csv(DATASET_PATH)
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [8]:
categorize_system_prompt = '''
Your goal is to extract movie categories from movie descriptions, as well as a 1-sentence summary for these movies.
You will be provided with a movie description, and you will output a json object containing the following information:

{
    categories: string[] // Array of categories based on the movie description,
    summary: string // 1-sentence summary of the movie based on the movie description
}

Categories refer to the genre or type of the movie, like "action", "romance", "comedy", etc. Keep category names simple and use only lower case letters.
Movies can have several categories, but try to keep it under 3-4. Only mention the categories that are the most obvious based on the description.
'''

def get_categories(description):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    temperature=0.1,
    # This is to enable JSON mode, making sure responses are valid json objects
    response_format={
        "type": "json_object"
    },
    messages=[
        {
            "role": "system",
            "content": categorize_system_prompt
        },
        {
            "role": "user",
            "content": description
        }
    ],
    )

    return response.choices[0].message.content

In [9]:
# Testing on a few examples
for _, row in df[:5].iterrows():
    description = row['Overview']
    title = row['Series_Title']
    result = get_categories(description)
    print(f"TITLE: {title}\nOVERVIEW: {description}\n\nRESULT: {result}")
    print("\n\n----------------------------\n\n")

TITLE: The Shawshank Redemption
OVERVIEW: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.

RESULT: {
    "categories": ["drama"],
    "summary": "Two imprisoned men bond over the years and find redemption through acts of common decency."
}


----------------------------


TITLE: The Godfather
OVERVIEW: An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.

RESULT: {
    "categories": ["crime", "drama"],
    "summary": "A crime drama about an aging patriarch passing on his empire to his son."
}


----------------------------


TITLE: The Dark Knight
OVERVIEW: When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.

RESULT: {
    "categories": ["action", "crime", "drama"],
    "summary": "Batman faces a psychological and physical 

In [10]:
# Creating an array of json tasks

tasks = []

for index, row in df.iterrows():

    description = row['Overview']

    task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # This is what you would have in your Chat Completions API call
            "model": "gpt-3.5-turbo",
            "temperature": 0.1,
            "response_format": {
                "type": "json_object"
            },
            "messages": [
                {
                    "role": "system",
                    "content": categorize_system_prompt
                },
                {
                    "role": "user",
                    "content": description
                }
            ],
        }
    }

    tasks.append(task)

In [11]:
# Creating the file

file_name = "batch_tasks_movies.jsonl"

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

In [12]:
batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)

In [13]:
print(batch_file)

FileObject(id='file-hCCNVHI2ZRw7I5tenVaczDVH', bytes=1127310, created_at=1720033807, filename='batch_tasks_movies.jsonl', object='file', purpose='batch', status='processed', status_details=None)


In [14]:
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [17]:
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_VdIl3yEHpkSTwVM6jPIcskjx', completion_window='24h', created_at=1720033864, endpoint='/v1/chat/completions', input_file_id='file-hCCNVHI2ZRw7I5tenVaczDVH', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1720120264, failed_at=None, finalizing_at=None, in_progress_at=1720033865, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=232, failed=0, total=1000))


In [18]:
import time

while True:
    batch_job = client.batches.retrieve(batch_job.id)
    print(batch_job)
    if batch_job.status == 'completed':
        break
    time.sleep(10)  # Wait for 10 seconds before checking again

Batch(id='batch_VdIl3yEHpkSTwVM6jPIcskjx', completion_window='24h', created_at=1720033864, endpoint='/v1/chat/completions', input_file_id='file-hCCNVHI2ZRw7I5tenVaczDVH', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1720120264, failed_at=None, finalizing_at=None, in_progress_at=1720033865, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=975, failed=0, total=1000))
Batch(id='batch_VdIl3yEHpkSTwVM6jPIcskjx', completion_window='24h', created_at=1720033864, endpoint='/v1/chat/completions', input_file_id='file-hCCNVHI2ZRw7I5tenVaczDVH', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1720120264, failed_at=None, finalizing_at=None, in_progress_at=1720033865, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=981, f

In [19]:
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

In [20]:
result_file_name = "batch_job_results_movies.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)

In [21]:
# Loading data from saved file
results = []
with open(result_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [22]:
# Reading only the first results
for res in results[:5]:
    task_id = res['custom_id']
    # Getting index from task id
    index = task_id.split('-')[-1]
    result = res['response']['body']['choices'][0]['message']['content']
    movie = df.iloc[int(index)]
    description = movie['Overview']
    title = movie['Series_Title']
    print(f"TITLE: {title}\nOVERVIEW: {description}\n\nRESULT: {result}")
    print("\n\n----------------------------\n\n")

TITLE: The Shawshank Redemption
OVERVIEW: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.

RESULT: {
    "categories": ["drama"],
    "summary": "Two imprisoned men bond over the years, finding solace and redemption through acts of decency."
}


----------------------------


TITLE: The Godfather
OVERVIEW: An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.

RESULT: {
    "categories": ["crime", "drama"],
    "summary": "A crime drama about an aging patriarch passing on his empire to his son."
}


----------------------------


TITLE: The Dark Knight
OVERVIEW: When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.

RESULT: {
    "categories": ["action", "crime", "drama"],
    "summary": "A dark and thrilling action-packed mov

In [23]:
DATASET_URL = "https://github.com/openai/openai-cookbook/raw/main/examples/data/amazon_furniture_dataset.csv"
response = requests.get(DATASET_URL)
DATASET_PATH = "amazon_furniture_dataset.csv"
with open(DATASET_PATH, 'wb') as file:
    file.write(response.content)

In [24]:
dataset_path = "amazon_furniture_dataset.csv"
df = pd.read_csv(dataset_path)
df.head()

,asin,url,title,brand,price,availability,categories,primary_image,images,upc,...,color,material,style,important_information,product_overview,about_item,description,specifications,uniq_id,scraped_at
0,B0CJHKVG6P,https://www.amazon.com/dp/B0CJHKVG6P,"GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...",GOYMFK,$24.99,Only 13 left in stock - order soon.,"['Home & Kitchen', 'Storage & Organization', '...",https://m.media-amazon.com/images/I/416WaLx10j...,['https://m.media-amazon.com/images/I/416WaLx1...,NaN,...,White,Metal,Modern,[],"[{'Brand': ' GOYMFK '}, {'Color': ' White '}, ...",['Multiple layers: Provides ample storage spac...,"multiple shoes, coats, hats, and other items E...","['Brand: GOYMFK', 'Color: White', 'Material: M...",02593e81-5c09-5069-8516-b0b29f439ded,2024-02-02 15:15:08
1,B0B66QHB23,https://www.amazon.com/dp/B0B66QHB23,"subrtex Leather ding Room, Dining Chairs Set o...",subrtex,NaN,NaN,"['Home & Kitchen', 'Furniture', 'Dining Room F...",https://m.media-amazon.com/images/I/31SejUEWY7...,['https://m.media-amazon.com/images/I/31SejUEW...,NaN,...,Black,Sponge,Black Rubber Wood,[],NaN,['【Easy Assembly】: Set of 2 dining room chairs...,subrtex Dining chairs Set of 2,"['Brand: subrtex', 'Color: Black', 'Product Di...",5938d217-b8c5-5d3e-b1cf-e28e340f292e,2024-02-02 15:15:09
2,B0BXRTWLYK,https://www.amazon.com/dp/B0BXRTWLYK,Plant Repotting Mat MUYETOL Waterproof Transpl...,MUYETOL,$5.98,In Stock,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",https://m.media-amazon.com/images/I/41RgefVq70...,['https://m.media-amazon.com/images/I/41RgefVq...,NaN,...,Green,Polyethylene,Modern,[],"[{'Brand': ' MUYETOL '}, {'Size': ' 26.8*26.8 ...","['PLANT REPOTTING MAT SIZE: 26.8"" x 26.8"", squ...",NaN,"['Brand: MUYETOL', 'Size: 26.8*26.8', 'Item We...",b2ede786-3f51-5a45-9a5b-bcf856958cd8,2024-02-02 15:15:09
3,B0C1MRB2M8,https://www.amazon.com/dp/B0C1MRB2M8,"Pickleball Doormat, Welcome Doormat Absorbent ...",VEWETOL,$13.99,Only 10 left in stock - order soon.,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",https://m.media-amazon.com/images/I/61vz1Igler...,['https://m.media-amazon.com/images/I/61vz1Igl...,NaN,...,A5589,Rubber,Modern,[],"[{'Brand': ' VEWETOL '}, {'Size': ' 16*24INCH ...","['Specifications: 16x24 Inch ', "" High-Quality...",The decorative doormat features a subtle textu...,"['Brand: VEWETOL', 'Size: 16*24INCH', 'Materia...",8fd9377b-cfa6-5f10-835c-6b8eca2816b5,2024-02-02 15:15:10
4,B0CG1N9QRC,https://www.amazon.com/dp/B0CG1N9QRC,JOIN IRON Foldable TV Trays for Eating Set of ...,JOIN IRON Store,$89.99,Usually ships within 5 to 6 weeks,"['Home & Kitchen', 'Furniture', 'Game & Recrea...",https://m.media-amazon.com/images/I/41p4d4VJnN...,['https://m.media-amazon.com/images/I/41p4d4VJ...,NaN,...,Grey Set of 4,Iron,X Classic Style,[],NaN,['Includes 4 Folding Tv Tray Tables And one Co...,Set of Four Folding Trays With Matching Storag...,"['Brand: JOIN IRON', 'Shape: Rectangular', 'In...",bdc9aa30-9439-50dc-8e89-213ea211d66a,2024-02-02 15:15:11


In [25]:
caption_system_prompt = '''
Your goal is to generate short, descriptive captions for images of items.
You will be provided with an item image and the name of that item and you will output a caption that captures the most important information about the item.
If there are multiple items depicted, refer to the name provided to understand which item you should describe.
Your generated caption should be short (1 sentence), and include only the most important information about the item.
The most important information could be: the type of item, the style (if mentioned), the material or color if especially relevant and/or any distinctive features.
Keep it short and to the point.
'''

def get_caption(img_url, title):
    response = client.chat.completions.create(
    model="gpt-4-turbo",
    temperature=0.2,
    max_tokens=300,
    messages=[
        {
            "role": "system",
            "content": caption_system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": title
                },
                # The content type should be "image_url" to use gpt-4-turbo's vision capabilities
                {
                    "type": "image_url",
                    "image_url": {
                        "url": img_url
                    }
                },
            ],
        }
    ]
    )

    return response.choices[0].message.content

In [26]:
# Testing on a few images
for _, row in df[:5].iterrows():
    img_url = row['primary_image']
    caption = get_caption(img_url, row['title'])
    img = Image(url=img_url)
    display(img)
    print(f"CAPTION: {caption}\n\n")

CAPTION: White multi-layer metal shoe rack with eight double hooks, featuring several shelves for shoes and additional storage space for accessories.




CAPTION: A set of two elegant black leather dining chairs with a simple vertical stitching detail on the backrest.




CAPTION: A green, waterproof, square, foldable plant repotting mat with raised edges, displayed with gardening tools and potted succulents, ideal for indoor gardening and easy cleanup.




CAPTION: Rectangular brown doormat featuring the phrase "It's a good day to play PICKLEBALL" with a pickleball paddle graphic, designed for absorbency and non-slip functionality.




CAPTION: Set of four foldable grey TV trays with a matching stand, featuring a sleek, space-saving design ideal for small living areas.




In [27]:
# Creating an array of json tasks

tasks = []

for index, row in df.iterrows():

    title = row['title']
    img_url = row['primary_image']

    task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # This is what you would have in your Chat Completions API call
            "model": "gpt-4-turbo",
            "temperature": 0.2,
            "max_tokens": 300,
            "messages": [
                {
                    "role": "system",
                    "content": caption_system_prompt
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": title
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": img_url
                            }
                        },
                    ],
                }
            ]
        }
    }

    tasks.append(task)

In [28]:
# Creating the file

file_name = "batch_tasks_furniture.jsonl"

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

In [29]:
# Uploading the file

batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)

In [30]:
# Creating the job

batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [31]:
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_0I6mxvi37100VHAfJu1uyjob', completion_window='24h', created_at=1720036444, endpoint='/v1/chat/completions', input_file_id='file-fO6GPGYI2PBsQbFwsb8PLIDS', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1720122844, failed_at=None, finalizing_at=None, in_progress_at=1720036445, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=312))


In [33]:
import time

while True:
    batch_job = client.batches.retrieve(batch_job.id)
    print(f"Status: {batch_job.status} - Completed: {batch_job.request_counts.completed}/{batch_job.request_counts.total}", end='\r')
    if batch_job.status == 'completed':
        break
    time.sleep(10)  # Wait for 10 seconds before checking again

print(f"\nBatch job completed with status: {batch_job.status}")

Status: finalizing - Completed: 312/312
Status: completed - Completed: 312/312

Batch job completed with status: completed


In [34]:
# Retrieving result file

result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

In [35]:
result_file_name = "batch_job_results_furniture.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)

In [36]:
# Loading data from saved file

results = []
with open(result_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [37]:
# Reading only the first results
for res in results[:5]:
    task_id = res['custom_id']
    # Getting index from task id
    index = task_id.split('-')[-1]
    result = res['response']['body']['choices'][0]['message']['content']
    item = df.iloc[int(index)]
    img_url = item['primary_image']
    img = Image(url=img_url)
    display(img)
    print(f"CAPTION: {result}\n\n")

CAPTION: White multi-layer metal shoe rack featuring eight double hooks for additional storage, ideal for organizing footwear and accessories in living spaces.




CAPTION: A set of two elegant black leather dining chairs with a simple vertical stitching detail on the backrest.




CAPTION: Green waterproof plant repotting mat, 26.8" x 26.8", featuring a portable and foldable design, ideal for indoor gardening and soil changing tasks.




CAPTION: A brown, absorbent non-slip doormat featuring the phrase "It's a good day to play PICKLEBALL" with a pickleball paddle design, measuring 16x24 inches.




CAPTION: Set of four foldable grey TV trays with a stand, featuring a compact and space-saving design, ideal for small spaces and convenient storage.


